In [ ]:
import polars as pl

from pathlib import Path

In [ ]:
RAW_DATA_PATH = Path("../data/raw_counter")
CLEAN_DATA_PATH = Path("../data/clean_counter")

YEAR_FILES = {
    2018: Path("2018_ecocounter_data_20230905140019.csv"),
    2019: Path("2019_ecocounter_data_20230905140408.csv"),
    2020: Path("2020_ecocounter_data_20230905140803.csv"),
    2021: Path("2021_ecocounter_data_20230905141200.csv"),
    2022: Path("2022_ecocounter_data_20230905141557.csv"),
    2023: Path("2023_ecocounter_data_20230905141951.csv"),
}

RENAME_MAPPING = {
    "Site ID": "site_id",
    "Site name": "site_name",
    "Date/time": "record_time",
    "Incoming count": "count_incoming",
    "Outgoing count": "count_outgoing",
}

**Data issues**
- "Count" columns are sometimes integer, sometimes string
- "Count" columns can contain nulls
- Current year contains future dates, prepopulated with Null
- Each (location,datetime) pair has two rows, for incoming and outgoing counts

**Cleaning**
1. Select incoming data rows where outgoing data is equal to zero. This eliminates null rows and those where the count is obviously in the other direction
2. Select the maximum count value per (location, datetime) pair
3. Repeat for outgoing data
4. Drop the incoming and outgoing data cols from the original dataframe, and uniquify the the rows to eliminate duplicate (location, datetime) pairs
5. Left going against the incoming and outgoing data rows.

This deduplicates the rows, ensuring each (location, datetime) occurs in exactly one row, and that the incoming and outgoing counts exist on the same row, while preserving the null information.

In [ ]:
for year, yf in YEAR_FILES.items():
    data = (
        pl.read_csv(RAW_DATA_PATH / yf)
        .with_columns(pl.col("^.* count$").cast(pl.Int64))
    )
    incoming_data = (
        data
        .filter(
            (pl.col("Incoming count").is_not_null()) & 
            (pl.col("Outgoing count") == 0)
        )
        .drop("Outgoing count")
        .lazy()
        .groupby(["Site ID", "Site name", "Date/time"])
        .max()
        .collect()
    )
    outgoing_data = (
        data
        .filter(
            (pl.col("Outgoing count").is_not_null()) & 
            (pl.col("Incoming count") == 0)
        )
        .drop("Incoming count")
        .lazy()
        .groupby(["Site ID", "Site name", "Date/time"])
        .max()
        .collect()
    )
    cleaned_data = (
        data
        .drop("Incoming count", "Outgoing count")
        .unique(maintain_order=True)
        .join(incoming_data, on=["Site ID", "Site name", "Date/time"], how="left")
        .join(outgoing_data, on=["Site ID", "Site name", "Date/time"], how="left")
        .rename(RENAME_MAPPING)
    )
    cleaned_data.write_csv(CLEAN_DATA_PATH / yf)


In [ ]:
cleaned_data

In [ ]:
cleaned_data = cleaned_data.with_columns(
    pl.col("record_time").str.to_datetime()
)

In [ ]:
cleaned_data.filter(
    (pl.col("record_time").dt.year() == 2022) &
    (pl.col("site_name") == "Basin Reserve") & 
    (pl.col("record_time").dt.month() == 8)
)